In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

salesOrderSchema = StructType([
     StructField("SalesOrderID", IntegerType()),
     StructField("OrderDate", StringType()),
     StructField("LineItemID", IntegerType()),
     StructField("CustomerID", IntegerType()),
     StructField("ProductID", IntegerType()),
     StructField("Quantity", IntegerType())
])
sales_order_raw = spark.read.format("csv").option("header", "true").load("/mnt/raw-data/salesorder_raw", schema=salesOrderSchema)

# Chuyển đổi cột Date thành kiểu dữ liệu ngày
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
sales_order_raw = sales_order_raw.withColumn("OrderDate", to_date(col("OrderDate"), "MM/dd/yyyy").cast(DateType()))

display(sales_order_raw)

In [ ]:
# Transform data
sales_order_raw.createOrReplaceTempView("sales_order_raw")
dim_sale_order = spark.sql("""
    SELECT SalesOrderID AS id,
           OrderDate AS order_date,
           Quantity AS quantity,
           CustomerID As dim_customer_id,
           ProductID As dim_product_id
    FROM sales_order_raw
""")
display(dim_sale_order)

In [ ]:
# dim_sale_order.write.format("com.databricks.spark.csv").option("header","true").option("delimiter", ",").mode("overwrite").save("/mnt/transformed-data/dim_sale_order")
dim_sale_order.write.format("delta").option("overwriteSchema", "true").mode("overwrite").save("/mnt/transformed-data/dim_sale_order")

In [ ]:
time_df = sales_order_raw.select(
    col("SalesOrderID").alias("id"),
    col("OrderDate").alias("date")
)

# Tính toán các thông tin về ngày/tháng/năm từ cột Date
time_df = time_df.withColumn("day_of_week", dayofweek(col("date")).cast(IntegerType()))
time_df = time_df.withColumn("day_of_month", dayofmonth(col("date")).cast(IntegerType()))
time_df = time_df.withColumn("week_of_year", weekofyear(col("date")).cast(IntegerType()))
time_df = time_df.withColumn("quarter", quarter(col("date")).cast(IntegerType()))
time_df = time_df.withColumn("month", month(col("date")).cast(IntegerType()))
time_df = time_df.withColumn("year", year(col("date")).cast(IntegerType()))

# Tạo bảng dim_date
dim_date = time_df.dropDuplicates(["date"]).orderBy("id")
dim_date.write.format("delta").mode("overwrite").save("/mnt/transformed-data/dim_date")

display(dim_date)